# Let's practice !
In this exercice, you will implement a vertical 2D jumper that must reach a height of 1 meter with one torque-actuated leg made of two segments.
The vertical degree of freedom actuation must be forced to zero, to mimic free flying conditions.
The lower segment of the leg must be constrained with a contact point in the first phase which will then be released in the second phase.
The main objective is to minimize torques during each phase.
The first phase will last for 0.2s with 40 nodes and the second will last for 0.4s with 40 nodes.

The motion will look like this :

![SegmentLocal](img/slider.gif)

In [ ]:
import numpy as np
from bioptim import *

In [ ]:
model = (
    TorqueBiorbdModel("models/Slider1Leg.bioMod", contact_types=[ContactType.RIGID_EXPLICIT]), 
    TorqueBiorbdModel("models/Slider1Leg.bioMod")
)
nq = model[0].nb_q
ntau = model[0].nb_tau

# This is your turn
n_shooting = 40, 40
final_time = 0.2, 0.4
tau_min, tau_max, tau_init = -200, 200, 0

objective_functions = ObjectiveList()
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_CONTROL, key="tau", weight=0.1, phase=0)
objective_functions.add(ObjectiveFcn.Lagrange.MINIMIZE_CONTROL, key="tau", weight=0.1, phase=1)

x_bounds = BoundsList()
x_bounds.add("q", bounds=model[0].bounds_from_ranges("q"), phase=0)
x_bounds.add("qdot", bounds=model[0].bounds_from_ranges("qdot"), phase=0)
x_bounds.add("q", bounds=model[1].bounds_from_ranges("q"), phase=1)
x_bounds.add("qdot", bounds=model[1].bounds_from_ranges("qdot"), phase=1)

x_bounds[0]["q"][:, 0] = [0, 3*np.pi / 8, -3*np.pi / 4]  # Start in demi-squat pose
x_bounds[0]["q"].min[0, -1] = 0  # Translation of the slider at end between 0...
x_bounds[0]["q"].max[0, -1] = 0.25  # ... and 0.25 m
x_bounds[0]["qdot"][:, 0] = 0  # Start at rest

x_bounds[1]["q"][0, -1] = 1  # End simulation at 1 m of vertical translation
x_bounds[1]["qdot"][0, -1] = 0  # without speed

# Give some limits to the torque
u_bounds = BoundsList()
u_bounds.add("tau", min_bound=[tau_min] * ntau, max_bound=[tau_max] * ntau, phase=0)
u_bounds.add("tau", min_bound=[tau_min] * ntau, max_bound=[tau_max] * ntau, phase=1)

# But remove the capability to actuate the translational joint (first DOF)
u_bounds[0]["tau"][0, :] = 0
u_bounds[1]["tau"][0, :] = 0

# Give some initial guess to q of the first phase, the rest is automatically declared to be zero
x_init = InitialGuessList()
x_init.add("q", [0, 3*np.pi / 8, -3*np.pi / 4], phase=0)

# Constraints
constraints = ConstraintList()
constraints.add(
    ConstraintFcn.TRACK_EXPLICIT_RIGID_CONTACT_FORCES,
    min_bound=0,
    max_bound=np.inf,
    node=Node.ALL_SHOOTING,
    contact_index=0,  # z axis
)

ocp = OptimalControlProgram(
    model,
    n_shooting,
    final_time,
    x_init=x_init,
    x_bounds=x_bounds,
    u_bounds=u_bounds,
    objective_functions=objective_functions,
    constraints=constraints,
    n_threads=8,
)

# have a look on the optimal control problem
ocp.print(to_console=False, to_graph=False)

In [ ]:
sol = ocp.solve()

In [ ]:
sol.print_cost()
sol.graphs(automatically_organize=False)

In [ ]:
# This cell for visualizing the animation won't work in a jupyter notebook, but would work locally, assuming 
# a viewer ([bioviz] or pyorerun) is properly installed

# sol.animate(n_frames=50)